In [132]:
from typing import Dict, List, Any, Hashable, Tuple
from datetime import datetime
import pandas as pd
import hashlib
import json

In [133]:
SINGLE_FIELDS = {
    # Identification
    "nct_id": "protocolSection.identificationModule.nctId",
    "brief_title": "protocolSection.identificationModule.briefTitle",
    "official_title": "protocolSection.identificationModule.officialTitle",
    "acronym": "protocolSection.identificationModule.acronym",
    "org_study_id": "protocolSection.identificationModule.orgStudyIdInfo.id",
    "org_study_type": "protocolSection.identificationModule.orgStudyIdInfo.type",
    "org_study_link": "protocolSection.identificationModule.orgStudyIdInfo.link",

    # Description
    "brief_summary": "protocolSection.descriptionModule.briefSummary",
    "detailed_desc": "protocolSection.descriptionModule.detailedDescription",

    # ResponsibleParty
    "responsible_party": "protocolSection.sponsorCollaboratorsModule.responsibleParty.type",

    # Design (single values)
    "study_type": "protocolSection.designModule.studyType",
    "patient_registry": "protocolSection.designModule.patientRegistry",
    "enrollment_type": "protocolSection.designModule.enrollmentInfo.type",
    "enrollment_count": "protocolSection.designModule.enrollmentInfo.count",
    "design_allocation": "protocolSection.designModule.designInfo.allocation",
    "design_intervention_model": "protocolSection.designModule.designInfo.interventionModel",
    "design_intervention_model_desc": "protocolSection.designModule.designInfo.interventionModelDescription",
    "design_primary_purpose": "protocolSection.designModule.designInfo.primaryPurpose",
    "design_observational_model": "protocolSection.designModule.designInfo.observationalModel",
    "design_time_perspective": "protocolSection.designModule.designInfo.timePerspective",
    "design_masking": "protocolSection.designModule.designInfo.maskingInfo.masking",
    "design_masking_desc": "protocolSection.designModule.designInfo.maskingInfo.maskingDescription",
    "design_who_masked": "protocolSection.designModule.designInfo.maskingInfo.whoMasked",

    # Expanded access
    "exp_acc_type_individual": "protocolSection.designModule.expandedAccessTypes.individual",
    "exp_acc_type_intermediate": "protocolSection.designModule.expandedAccessTypes.intermediate",
    "exp_acc_type_treatment": "protocolSection.designModule.expandedAccessTypes.treatment",

    # Biospecimen
    "biospec_retention": "protocolSection.designModule.bioSpec.retention",
    "biospec_desc": "protocolSection.designModule.bioSpec.description",

    # Eligibility
    "eligibility_criteria": "protocolSection.eligibilityModule.eligibilityCriteria",
    "healthy_volunteers": "protocolSection.eligibilityModule.healthyVolunteers",
    "sex": "protocolSection.eligibilityModule.sex",
    "gender_based": "protocolSection.eligibilityModule.genderBased",
    "gender_desc": "protocolSection.eligibilityModule.genderDescription",
    "min_age": "protocolSection.eligibilityModule.minimumAge",
    "max_age": "protocolSection.eligibilityModule.maximumAge",
    "population_desc": "protocolSection.eligibilityModule.studyPopulation",
    "sampling_method": "protocolSection.eligibilityModule.samplingMethod",

    # Status
    "overall_status": "protocolSection.statusModule.overallStatus",
    "last_known_status": "protocolSection.statusModule.lastKnownStatus",
    "status_verified_date": "protocolSection.statusModule.statusVerifiedDate",
    "delayed_posting": "protocolSection.statusModule.delayedPosting",
    "start_date": "protocolSection.statusModule.startDateStruct.date",
    "start_date_type": "protocolSection.statusModule.startDateStruct.type",
    "first_submit_date": "protocolSection.statusModule.studyFirstSubmitDate",
    "first_submit_qc_date": "protocolSection.statusModule.studyFirstSubmitQcDate",
    "last_update_submit_date": "protocolSection.statusModule.lastUpdateSubmitDate",
    "completion_date": "protocolSection.statusModule.completionDateStruct.date",
    "completion_date_type": "protocolSection.statusModule.completionDateStruct.type",
    "why_stopped": "protocolSection.statusModule.whyStopped",
    "has_expanded_access": "protocolSection.statusModule.expandedAccessInfo.hasExpandedAccess",

    # Oversight
    "has_dmc": "protocolSection.oversightModule.oversightHasDmc",
    "is_fda_regulated_drug": "protocolSection.oversightModule.isFdaRegulatedDrug",
    "is_fda_regulated_device": "protocolSection.oversightModule.isFdaRegulatedDevice",
    "is_unapproved_device": "protocolSection.oversightModule.isUnapprovedDevice",
    "is_us_export": "protocolSection.oversightModule.isUsExport",

    # Individual participant data
    "ipd_sharing": "protocolSection.ipdSharingStatementModule.ipdSharing",
    "ipd_desc": "protocolSection.ipdSharingStatementModule.description",
    "ipd_time_frame": "protocolSection.ipdSharingStatementModule.timeFrame",
    "ipd_access_criteria": "protocolSection.ipdSharingStatementModule.accessCriteria",
    "ipd_url": "protocolSection.ipdSharingStatementModule.url",

    # Large documents
    "large_doc_no_sap": "documentSection.largeDocumentModule.noSap",

    # Miscellaneous
    "version_holder": "derivedSection.miscInfoModule.versionHolder",
    "has_results": "hasResults",
    "last_updated": "protocolSection.statusModule.lastUpdatePostDateStruct.date",
    "unposted_responsible_party": "annotationSection.annotationModule.unpostedAnnotation.unpostedResponsibleParty",
    "limitations_desc": "resultsSection.moreInfoModule.limitationsAndCaveats.description",

    # Certain agreements
    "certain_agreement_pi_sponsor_employee": "resultsSection.moreInfoModule.certainAgreement.piSponsorEmployee",
    "certain_agreement_restriction_type": "resultsSection.moreInfoModule.certainAgreement.restrictionType",
    "certain_agreement_restrictive": "resultsSection.moreInfoModule.certainAgreement.restrictiveAgreement",
    "certain_agreement_other_details": "resultsSection.moreInfoModule.certainAgreement.otherDetails",

    # Point of contact
    "poc_title": "resultsSection.moreInfoModule.pointOfContact.title",
    "poc_organization": "resultsSection.moreInfoModule.pointOfContact.organization",
    "poc_email": "resultsSection.moreInfoModule.pointOfContact.email",
    "poc_phone": "resultsSection.moreInfoModule.pointOfContact.phone",
    "poc_phone_ext": "resultsSection.moreInfoModule.pointOfContact.phoneExt",

    # Submission tracking
    "sub_tracking_estimated_results_date": "derivedSection.miscInfoModule.submissionTracking.estimatedResultsFirstSubmitDate",
    "sub_tracking_first_mcp_date": "derivedSection.miscInfoModule.submissionTracking.firstMcpInfo.postDateStruct.date",
    "sub_tracking_first_mcp_type": "derivedSection.miscInfoModule.submissionTracking.firstMcpInfo.postDateStruct.type",
}

NESTED_FIELDS = {

   "sponsors": {
        "index_field": "protocolSection.sponsorCollaboratorsModule.leadSponsor",
        "object_type": "simple dict",
        "fields": [
            ("lead_sponsor_name", "name"),
            ("lead_sponsor_class", "class"),
        ],
        "table_name": "sponsors",
        "bridge_table_name": "study_sponsors",
        "transformer_method": "extract_sponsors"
    },

    "collaborators": {
        "index_field": "protocolSection.sponsorCollaboratorsModule.collaborator",
        "object_type": "array_of_dicts",
        "fields": [
            ("sponsor_name", "name"),
            ("sponsor_class", "class"),
        ],
        "table_name": "sponsor",
        "bridge_table_name": "study_sponsors",
        "transformer_method": "extract_sponsors"
    },
    
    "conditions": {
        "index_field": "protocolSection.conditionsModule.conditions",
        "object_type": "simple_array",
        "table_name": "conditions",
        "bridge_table_name": "bridge_study_conditions",
        "field_name": "condition_name",
        "transformer_method":"extract_conditions"
    },
    
    "keywords": {
        "index_field": "protocolSection.conditionsModule.keywords",
        "object_type": "simple_array",
        "table_name": "keywords",
        "bridge_table_name": "bridge_study_keywords",
        "field_name": "keyword",
        "transformer_method":"extract_keywords"
    },

    "arm_groups": {
        "index_field": "protocolSection.armsInterventionsModule.armGroups",
        "object_type": "array_of_dicts",
        "table_name": "study_arm_groups",
        "fields": [
            ("arm_group_label", "label"),
            ("arm_group_type", "type"),
            ("arm_group_desc", "description")
        ],
        "nested": {
            "interventionNames": {
                "object_type": "nested_simple_array",
                "bridge_table_name": "arm_group_interventions",
                "field_name": "intervention_name"
        },
        "transformer_method": "extract_arm_groups"
        }
    },


    "interventions": {
        "index_field": "protocolSection.armsInterventionsModule.interventions",
        "object_type": "array_of_dicts",
        "fields": [
            ("intervention_name", "name"),
            ("intervention_desc", "description"),
            ("intervention_type", "type"),
        ],
        "nested": {
            "otherNames": {
                "object_type": "nested_simple_array",
                "table_name": "interventions",
                "bridge_table_name": "bridge_table_name"
            }

        },
        "table_name": "interventions",
        "bridge_table_name": "bridge_study_interventions",
        "transformer_method": "extract_interventions"

    },


    "locations": {
        "index_field": "protocolSection.contactsLocationsModule.locations",
        "object_type": "array_of_dicts",
        "table_name": "sites",
        "bridge_table_name": "study_sites",
        "fields": [
            ("site_facility", "facility"),
            ("city", "city"),
            ("state", "state"),
            ("zip", "zip"),
            ("country", "country"),
            ("site_status", "status"),
        ],
        "nested": {
            "geoPoint": {
                "object_type": "simple_dict",
                 "fields": ["lat", "lon"],
            },

            "contacts": {
                "object_type": "nested_array_of_dicts",
                "table_name": "contacts",
                "bridge_table_name": "location_contacts",
                "fields": [
                    ("name", "name"),
                    ("role", "role"),
                    ("email", "email"),
                    ("phone", "phone"),
                    ("phoneExt", "phoneExt")
                ],
                "transformer_method": "extract_contacts"
            }
        },
        "transformer_method": "extract_locations"
    },

    "central_contacts": {
        "index_field": "protocolSection.contactsLocationsModule.centralContacts",
        "object_type": "array_of_dicts",
        "table_name": "contacts",
        "bridge_table_name": "study_contacts",
        "fields": [
            ("name", "name"),
            ("role", "role"),
            ("email", "email"),
            ("phone", "phone"),
            ("phoneExt", "phoneExt")
        ],
        "transformer_method": "extract_contacts"
    },

    "overall_officials": {
        "index_field": "protocolSection.contactsLocationsModule.overallOfficials",
        "object_type": "array_of_dicts",
        "table_name": "investigators",
        "bridge_table_name": "study_investigators",
        "fields": [
            ("name", "name"),
            ("affiliation", "affiliation"),
            ("role", "role")
        ],
        "transformer_method": "extract_officials"
    },

    "primary_outcomes": {
        "index_field": "protocolSection.outcomesModule.primaryOutcomes",
        "object_type": "array_of_dicts",
        "table_name": "study_outcomes",
        "fields": [
            ("measure", "measure"),
            ("description", "description"),
            ("timeFrame", "timeFrame")
        ],
        "transformer_method": "extract_outcomes",
        "outcome_type": "PRIMARY",
    },

    "secondary_outcomes": {
        "index_field": "protocolSection.outcomesModule.secondaryOutcomes",
        "object_type": "array_of_dicts",
        "table_name": "study_outcomes",
        "fields": [
            ("measure", "measure"),
            ("description", "description"),
            ("timeFrame", "timeFrame")
        ],
        "transformer_method": "extract_outcomes",
        "outcome_type": "SECONDARY",
    },

    "other_outcomes": {
        "index_field": "protocolSection.outcomesModule.otherOutcomes",
        "object_type": "array_of_dicts",
        "table_name": "study_outcomes",
        "fields": [
            ("measure", "measure"),
            ("description", "description"),
            ("timeFrame", "timeFrame")
        ],
        "transformer_method": "extract_outcomes",
        "outcome_type": "OTHER",
    },

    "references": {
        "index_field": "protocolSection.referencesModule.references",
        "object_type": "array_of_dicts",
        "table_name": "study_publications",
        "extract_fields": ["pmid", "object_type", "citation"],
    },

    "retractions": {
        "index_field": "protocolSection.referencesModule.retractions",
        "object_type": "array_of_dicts",
        "table_name": "study_retractions",
        "fields": [
            ("pmid", "pmid"),
            ("status", "status")
        ],
        "transformer_method": "extract_outcomes",
    },

    "see_also": {
        "index_field": "protocolSection.referencesModule.seeAlsoLinks",
        "object_type": "array_of_dicts",
        "table_name": "study_see_also",
        "fields": [
            ("label", "label"),
            ("url", "url")
        ],
        "transformer_method": "extract_see_also"
    },

    "phases": {
        "index_field": "protocolSection.designModule.phases",
        "object_type": "simple_array",
        "table_name": "phases",
        "bridge_table_name": "study_phases",
        "field_name": "phase",
        "transformer_method": "extract_study_phases"
    },


    "std_ages": {
        "index_field": "protocolSection.eligibilityModule.stdAges",
        "object_type": "simple_array",
        "table_name": "age_groups",
        "bridge_table_name": "study_age_groups",
        "field_name": "age_group",
        "transformer_method": "extract_age_group"
    },

    "ipd_info_types": {
        "index_field": "protocolSection.ipdSharingStatementModule.infoTypes",
        "object_type": "simple_array",
        "table_name": "ipd_info_types",
        "bridge_table_name": "study_ipd_info_types",
        "field_name": "info_type",
        "transformer_method": "extract_ipd_info_types"
    },

    "secondary_id_infos": {
        "index_field": "protocolSection.identificationModule.secondaryIdInfos",
        "object_type": "array_of_dicts",
        "table_name": "secondary_ids",
        "bridge_table_name": "study_secondary_ids",
        "fields": [
            ("id", "id"),
            ("object_type", "object_type"),
            ("domain", "domain"),
            ("link", "link")
        ],
        "transformer_method": "extract_id_infos"

    },
    "nct_id_aliases": {
        "index_field": "protocolSection.identificationModule.nctIdAliases",
        "object_type": "simple_array",
        "table_name": "nct_aliases",
        "bridge_table_name": "study_nct_aliases",
        "field_name": "alias_nct_id",
        "transformer_method": "extract_nct_id_aliases"
    },

    # ===== DERIVED SECTION (MeSH) =====
    # CONDITION MESH TERMS


    "condition_mesh_terms": {
        "index_field": "derivedSection.conditionBrowseModule.meshes",
        "object_type": "array_of_dicts",
        "fields": [
            ("id", "id"),
            ("term", "term")
        ],
        "table_name": "condition_mesh_terms",
        "bridge_table_name": "study_conditions_mesh",
        "is_primary": True,
        "transformer_method": "extract_condition_mesh"
    },

    "condition_mesh_ancestors": {
        "index_field": "derivedSection.conditionBrowseModule.ancestors",
        "object_type": "array_of_dicts",
        "fields": [
            ("id", "id"),
            ("term", "term")
        ],
        "table_name": "condition_mesh_terms",
        "bridge_table_name": "study_conditions_mesh",
        "is_primary": False,
        "transformer_method": "extract_condition_mesh"
    },

    "intervention_mesh_terms": {
        "index_field": "derivedSection.interventionBrowseModule.meshes",
        "object_type": "array_of_dicts",
        "fields": [
            ("id", "id"),
            ("term", "term")
        ],
        "table_name": "intervention_mesh_terms",
        "bridge_table_name": "study_interventions_mesh",
        "is_primary": True,
        "transformer_method": "extract_intervention_mesh"
    },

    "intervention_mesh_ancestors": {
        "index_field": "derivedSection.interventionBrowseModule.ancestors",
        "object_type": "array_of_dicts",
        "fields": [
            ("id", "id"),
            ("term", "term")
        ],
        "table_name": "intervention_mesh_terms",
        "bridge_table_name": "study_interventions_mesh",
        "is_primary": False,
        "transformer_method": "extract_intervention_mesh"
    },

    "large_documents": {
        "index_field": "documentSection.largeDocumentModule.largeDocs",
        "object_type": "array_of_dicts",
        "fields": [
            ("typeAbbrev", "typeAbbrev"),
            ("hasProtocol", "hasProtocol"),
            ("hasSap", "hasSap"),
            ("hasIcf", "hasIcf"),
            ("label", "label"),
            ("date", "date"),
            ("uploadDate", "uploadDate"),
            ("filename", "filename"),
            ("size", "size"),
        ],
        "table_name": "study_documents",
        "transformer_method": "extract_large_documents"
    },

    "unposted_events": {
        "index_field": "annotationSection.annotationModule.unpostedAnnotation.unpostedEvents",
        "object_type": "array_of_dicts",
        "fields": [
            ("object_type", "object_type"),
            ("date", "date"),
            ("dateUnknown", "dateUnknown"),
        ],
        "table_name": "unposted_events",
        "bridge_table_name": "study_unposted_events",
        "transformer_method": "extract_unposted_events"
    },

    "violation_events": {
        "index_field": "annotationSection.annotationModule.violationAnnotation.violationEvents",
        "object_type": "array_of_dicts",
        "fields": [
            ("object_type", "object_type"),
            ("description", "description"),
            ("creationDate", "creationDate"),
            ("issuedDate", "issuedDate"),
            ("releaseDate", "releaseDate"),
            ("postedDate", "postedDate"),
        ],
        "table_name": "violation_events",
        "bridge_table_name": "study_violation_events",
        "transformer_method": "extract_violation_events"
    },

    "removed_countries": {
        "index_field": "derivedSection.miscInfoModule.removedCountries",
        "object_type": "simple_array",
        "table_name": "countries",
        "bridge_table_name": "study_removed_countries",
        "field_name": "country",
        "transformer_method": "extract_removed_countries"
    },

    "sub_infos": {
        "index_field": "derivedSection.miscInfoModule.submissionTracking.submissionInfos",
        "object_type": "array_of_dicts",
        "fields": [
            ("releaseDate", "releaseDate"),
            ("unreleaseDate", "unreleaseDate"),
            ("unreleaseDateUnknown", "unreleaseDateUnknown"),
            ("resetDate", "resetDate"),
            ("mcpReleaseN", "mcpReleaseN")
        ],
        "table_name": "sub_tracking",
        "bridge_table_name": "study_sub_tracking",
        "transformer_method": "extract_sub_infos"
    },


    # # PARTICIPANT FLOW GROUPS
    # 'participant_flow_groups': {
    #     'path': 'resultsSection.participantFlowModule.groups',
    #     'type': 'array_of_dicts',
    #     'table_name': 'flow_groups',
    #     'bridge_table_name': 'study_flow_groups',
    #     'extract_fields': ['id', 'title', 'description']
    # },
    #
    # # PARTICIPANT FLOW PERIODS
    # 'participant_flow_periods': {
    #     'path': 'resultsSection.participantFlowModule.periods',
    #     'type': 'array_of_dicts',
    #     'table_name': 'flow_periods',
    #     'bridge_table_name': 'study_flow_periods',
    #     'extract_fields': ['title'],
    #     'nested_arrays': {
    #         'milestones': ['type', 'comment', 'achievements'],
    #         'dropWithdraws': ['type', 'comment', 'reasons']
    #     }
    # },
    #
    # # BASELINE GROUPS
    # 'baseline_groups': {
    #     'path': 'resultsSection.baselineCharacteristicsModule.groups',
    #     'type': 'array_of_dicts',
    #     'table_name': 'baseline_groups',
    #     'bridge_table_name': 'study_baseline_groups',
    #     'extract_fields': ['id', 'title', 'description']
    # },
    #
    # # BASELINE DENOMS
    # 'baseline_denoms': {
    #     'path': 'resultsSection.baselineCharacteristicsModule.denoms',
    #     'type': 'array_of_dicts',
    #     'table_name': 'baseline_denoms',
    #     'extract_fields': ['units'],
    #     'nested_arrays': {
    #         'counts': ['groupId', 'value']
    #     }
    # },
    #
    # # BASELINE MEASURES
    # 'baseline_measures': {
    #     'path': 'resultsSection.baselineCharacteristicsModule.measures',
    #     'type': 'array_of_dicts',
    #     'table_name': 'baseline_measures',
    #     'extract_fields': [
    #         'title', 'description', 'populationDescription',
    #         'paramType', 'dispersionType', 'unitOfMeasure',
    #         'calculatePct', 'denomUnitsSelected'
    #     ],
    #     'nested_arrays': {
    #         'denoms': ['units', 'counts'],
    #         'classes': ['title', 'denoms', 'categories']
    #     }
    # },
    #
    # # OUTCOME MEASURES
    # 'outcome_measures_results': {
    #     'path': 'resultsSection.outcomeMeasuresModule.outcomeMeasures',
    #     'type': 'array_of_dicts',
    #     'table_name': 'outcome_measure_results',
    #     'extract_fields': [
    #         'type', 'title', 'description', 'populationDescription',
    #         'reportingStatus', 'anticipatedPostingDate',
    #         'paramType', 'dispersionType', 'unitOfMeasure',
    #         'calculatePct', 'timeFrame', 'typeUnitsAnalyzed',
    #         'denomUnitsSelected'
    #     ],
    #     'nested_arrays': {
    #         'groups': ['id', 'title', 'description'],
    #         'denoms': ['units', 'counts'],
    #         'classes': ['title', 'denoms', 'categories'],
    #         'analyses': [
    #             'groupIds', 'paramType', 'paramValue',
    #             'dispersionType', 'dispersionValue',
    #             'statisticalMethod', 'statisticalComment',
    #             'pValue', 'pValueComment',
    #             'ciNumSides', 'ciPctValue', 'ciUpperLimit', 'ciLowerLimit',
    #             'estimateComment', 'testedNonInferiority',
    #             'nonInferiorityType', 'nonInferiorityComment',
    #             'otherAnalysisDescription', 'groupDescription'
    #         ]
    #     }
    # },
    #
    # # ADVERSE EVENT GROUPS
    # 'adverse_event_groups': {
    #     'path': 'resultsSection.adverseEventsModule.eventGroups',
    #     'type': 'array_of_dicts',
    #     'table_name': 'ae_groups',
    #     'bridge_table_name': 'study_ae_groups',
    #     'extract_fields': [
    #         'id', 'title', 'description',
    #         'deathsNumAffected', 'deathsNumAtRisk',
    #         'seriousNumAffected', 'seriousNumAtRisk',
    #         'otherNumAffected', 'otherNumAtRisk'
    #     ]
    # },
    #
    # # SERIOUS ADVERSE EVENTS
    # 'serious_adverse_events': {
    #     'path': 'resultsSection.adverseEventsModule.seriousEvents',
    #     'type': 'array_of_dicts',
    #     'table_name': 'adverse_events',
    #     'extract_fields': [
    #         'term', 'organSystem', 'sourceVocabulary',
    #         'assessmentType', 'notes'
    #     ],
    #     'nested_arrays': {
    #         'stats': ['groupId', 'numEvents', 'numAffected', 'numAtRisk']
    #     },
    #     'severity': 'SERIOUS'
    # },
    #
    # # OTHER ADVERSE EVENTS
    # 'other_adverse_events': {
    #     'path': 'resultsSection.adverseEventsModule.otherEvents',
    #     'type': 'array_of_dicts',
    #     'table_name': 'adverse_events',
    #     'extract_fields': [
    #         'term', 'organSystem', 'sourceVocabulary',
    #         'assessmentType', 'notes'
    #     ],
    #     'nested_arrays': {
    #         'stats': ['groupId', 'numEvents', 'numAffected', 'numAtRisk']
    #     },
    #     'severity': 'OTHER'
    # }

    }


In [134]:
def generate_key(*args) -> str:
    """Generates a deterministic surrogate key from input values."""
    combined = "|".join(str(arg) for arg in args if arg is not None)
    return hashlib.sha256(combined.encode()).hexdigest()[:16]

In [150]:
def deep_get(data, path: str):
    """Navigate nested dict using dot separated path, or flat Series/dict."""
    #for normalised dfs
    if isinstance(data, pd.Series):
        return data.get(path)
    
    # dict navigation
    keys = path.split('.')
    value = data
    for key in keys:
        if isinstance(value, dict):
            value = value.get(key)
        else:
            return None
    return value

In [151]:
def extract_study_fields(study_key: str, study_df: dict) -> Dict:
    study_record = dict()

    study_record['study_key'] = study_key
    for entity_key in SINGLE_FIELDS:
        index_field = SINGLE_FIELDS.get(entity_key)

        study_record[entity_key] = deep_get(study_df, index_field)

    return study_record

In [152]:
all_studies = []
all_sponsors = []
all_study_sponsors = []

all_conditions = []
all_study_conditions = []

all_keywords = []
all_study_keywords = []

all_study_arm_groups = []
all_study_arm_group_interventions = []

all_interventions = []
all_interventions_other_names = []
all_study_interventions = []

df = pd.read_parquet("page1.parquet")
df_studies = pd.json_normalize(df['studies'].tolist())


for idx, study in df_studies.iterrows():
    nct_id = deep_get(study, 'protocolSection.identificationModule.nctId')
    print(nct_id)
    if not nct_id:
        print(f"Study missing NCT ID, skipping {idx}") 
        continue
    study_key = generate_key(nct_id)


#     #study
#     study_record = extract_study_fields(study_key, study)
#     all_studies.append(study_record)

    # #sponsors
    # sponsors, study_sponsors = extract_sponsors(study_key, study)
    # all_sponsors.extend(sponsors)
    # all_study_sponsors.extend(study_sponsors)

    # #conditions and keywords
    # conditions, study_conditions = extract_conditions(study_key, study)
    # all_conditions.extend(conditions)
    # all_study_conditions.extend(study_conditions)

    # keywords, study_keywords = extract_keywords(study_key, study)
    # all_keywords.extend(keywords)
    # all_study_keywords.extend(study_keywords)

    # #groups and interventions
    # arm_groups, arm_group_interventions = extract_arm_groups(study_key, study)
    # all_study_arm_groups.extend(arm_groups)
    # all_study_arm_group_interventions.extend(arm_group_interventions)

    # interventions, intervention_other_names, study_interventions = extract_interventions(study_key, study)
    # all_interventions.extend(interventions)
    # all_interventions_other_names.extend(intervention_other_names)
    # all_study_interventions.extend(study_interventions)




#build dataframes from lists

# df_studies = pd.DataFrame(all_studies)

# df_sponsors = pd.DataFrame(all_sponsors)
# df_study_sponsors = pd.DataFrame(all_study_sponsors)


# df_conditions = pd.DataFrame(all_conditions)
# df_study_conditions=pd.DataFrame(all_study_conditions)

# df_keywords = pd.DataFrame(all_keywords)
# df_study_keywords = pd.DataFrame(all_study_keywords)

# df_study_arm_groups = pd.DataFrame(all_study_arm_groups)
# df_study_arm_group_interventions = pd.DataFrame(all_study_arm_group_interventions)

# df_interventions = pd.DataFrame(all_interventions)
# df_intervention_other_names = pd.DataFrame(all_interventions_other_names)
# df_study_interventions = pd.DataFrame(all_study_interventions)

# # dedupe
# df_sponsors = df_sponsors.drop_duplicates(subset=['sponsor_key'])
# df_conditions = df_conditions.drop_duplicates(subset=['condition_key'])
# df_keywords = df_keywords.drop_duplicates(subset=['keyword_key'])
# df_interventions = df_interventions.drop_duplicates(subset=['intervention_key'])
# df_intervention_other_names = df_intervention_other_names.drop_duplicates(subset=['intervention_key', 'other_name'])

NCT07130435
NCT03927235
NCT02645435
NCT01933035
NCT00457535
NCT03240835
NCT04713735
NCT04445935
NCT05491135
NCT05064735
NCT03012035
NCT04390035
NCT01578135
NCT04204135
NCT04007835
NCT02583035
NCT03980535
NCT07053735
NCT02285335
NCT06910735
NCT04403035
NCT01340235
NCT06817135
NCT05489835
NCT01644435
NCT00931008
NCT01409408
NCT06760208
NCT00404508
NCT03879408
NCT03948308
NCT02121808
NCT01890408
NCT00687908
NCT05127408
NCT06859008
NCT04049708
NCT04214808
NCT05210608
NCT06309108
NCT00343408
NCT02751008
NCT07111208
NCT05535608
NCT06462508
NCT04919408
NCT04109508
NCT04291508
NCT07123025
NCT01738425
NCT03345225
NCT05063825
NCT02001025
NCT03319225
NCT03146325
NCT02765425
NCT00372125
NCT05634525
NCT02274025
NCT03258125
NCT05964725
NCT05852925
NCT04734925
NCT02359825
NCT00788125
NCT00209625
NCT04387825
NCT04090125
NCT04165876
NCT04034576
NCT03492476
NCT07163676
NCT06451276
NCT06136676
NCT01346176
NCT00671476
NCT05283876
NCT00780676
NCT05490576
NCT04685876
NCT06615076
NCT06511076
NCT06772376
NCT0

In [138]:
print(df_studies)

     annotationSection  documentSection  hasResults  resultsSection  \
0                  NaN              NaN       False             NaN   
1                  NaN              NaN       False             NaN   
2                  NaN              NaN       False             NaN   
3                  NaN              NaN       False             NaN   
4                  NaN              NaN       False             NaN   
..                 ...              ...         ...             ...   
995                NaN              NaN        True             NaN   
996                NaN              NaN       False             NaN   
997                NaN              NaN       False             NaN   
998                NaN              NaN       False             NaN   
999                NaN              NaN        True             NaN   

     derivedSection.conditionBrowseModule  \
0                                     NaN   
1                                     NaN   
2           

In [119]:
df = pd.read_parquet("page1.parquet")

In [149]:
print(type(df_studies))


<class 'pandas.core.frame.DataFrame'>
